<a href="https://colab.research.google.com/github/Aamna-Khan-Git/Models/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUSTOMER SEGMENTATION PROJECT - streamlit**

## **SUBMITTED BY : AAMNA KHAN**
## Github link : [customer-segmentation](https://github.com/Aamna-Khan-Git/Models.git)

In [ ]:
# -------------------------------
# 📦 Imports
# -------------------------------
import streamlit as st
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# -------------------------------
# 📂 Load Models and Data
# -------------------------------
model = joblib.load("rfm_cluster_model.pkl")  # Customer segmentation model
item_similarity_df = joblib.load("item_similarity_matrix.pkl")  # Item similarity matrix

# Load the dataset for product mapping
d3 = pd.read_csv("online_retail.csv")

product_map = d3[['StockCode', 'Description']].drop_duplicates().dropna()
product_map = product_map.set_index('Description')

# -------------------------------
# 🔍 Helper: Find matching stock codes by product name
# -------------------------------
def find_stockcodes_by_name(product_name):
    matches = d3[d3['Description'].str.contains(product_name, case=False, na=False)]
    results = matches[['StockCode', 'Description']].drop_duplicates().reset_index(drop=True)
    return results if not results.empty else None

# -------------------------------
# 🎯 Product Recommendation Logic (Exact name match)
# -------------------------------
def get_recommendations_by_name(product_name, n=5):
    try:
        product_code = product_map.loc[product_name]['StockCode']
    except:
        return f"Product '{product_name}' not found.", None

    if product_code not in item_similarity_df.columns:
        return f"No similarity data found for '{product_name}'", None

    similarities = item_similarity_df[product_code].sort_values(ascending=False)
    top_codes = similarities.iloc[1:n+1].index.tolist()
    top_scores = similarities.iloc[1:n+1].values.round(3)

    # Map to product names
    code_to_name = d3.drop_duplicates(subset='StockCode').set_index('StockCode')['Description'].to_dict()
    top_names = [code_to_name.get(code, "(name not found)") for code in top_codes]

    result = pd.DataFrame({
        'Product Name': top_names,
        'StockCode': top_codes,
        'Similarity Score': top_scores
    })
    return None, result

# -------------------------------
# 🧠 Customer Segmentation Logic
# -------------------------------
def predict_cluster(recency, frequency, monetary):
    features = pd.DataFrame([[recency, frequency, monetary]],
                            columns=['Recency_scaled', 'Frequency_scaled', 'Monetary_scaled'])
    cluster = model.predict(features)[0]
    label_map = {
        0: "High-Value",
        1: "Regular",
        2: "Occasional",
        3: "At-Risk"
    }
    return label_map.get(cluster, "Unknown")

# -------------------------------
# 🖼️ Streamlit App UI
# -------------------------------
st.set_page_config(layout="wide")
st.title("🛍️ Retail Analytics Dashboard")

menu = st.sidebar.radio("Navigation", ["🧠 Customer Segmentation", "🎯 Product Recommendation"])

# -------------------------------
# 🎯 Product Recommendation UI
# -------------------------------
if menu == "🎯 Product Recommendation":
    st.subheader("🔍 Product Recommendation Module (Exact Match)")

    user_input = st.text_input("Enter an exact product name", placeholder="e.g. WHITE METAL LANTERN")
    if st.button("Get Recommendations"):
        error, recommendations = get_recommendations_by_name(user_input.upper())
        if error:
            st.warning(error)
        else:
            st.success("Top 5 Similar Products:")
            for idx, row in recommendations.iterrows():
                st.markdown(f"**{row['Product Name']}** — Similarity: {row['Similarity Score']}")

    st.subheader("🔍 Product Recommendation Module (Fuzzy Match)")
    input_name = st.text_input("Enter part of a product name", placeholder="e.g. lantern")

    if st.button("Search Product"):
        matches = find_stockcodes_by_name(input_name)

        if matches is None:
            st.warning(f"No products found containing '{input_name}'.")

        elif len(matches) == 1:
            selected_code = matches.loc[0, 'StockCode']
            st.success(f"Found 1 match: {matches.loc[0, 'Description']} (StockCode: {selected_code})")

            _, recommendations = get_recommendations_by_name(matches.loc[0, 'Description'])
            st.dataframe(recommendations)

        else:
            display_names = matches['Description'] + " (" + matches['StockCode'] + ")"
            selected = st.selectbox("Select a product:", display_names)
            selected_code = matches.loc[display_names == selected, 'StockCode'].values[0]
            selected_name = matches.loc[display_names == selected, 'Description'].values[0]

            _, recommendations = get_recommendations_by_name(selected_name)
            st.success(f"Showing recommendations for: {selected}")
            st.dataframe(recommendations)

# -------------------------------
# 🧠 Customer Segmentation UI
# -------------------------------
elif menu == "🧠 Customer Segmentation":
    st.subheader("📊 Customer Segmentation Module")

    recency = st.number_input("Recency (days since last purchase)", min_value=0, max_value=1000, value=30)
    frequency = st.number_input("Frequency (number of purchases)", min_value=1, max_value=500, value=5)
    monetary = st.number_input("Monetary (total spend)", min_value=1, max_value=100000, value=500)

    if st.button("Predict Cluster"):
        segment = predict_cluster(recency, frequency, monetary)
        st.success(f"This customer belongs to: **{segment} Shopper**")
